## Support Vector Machine

In [7]:
from src.svm_mod import SVM_model
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import src.utils as utils

In [8]:
df = pd.read_csv("data/diamonds.csv")
X, y, scaler = utils.preprocessing_no_drop(df)
X.shape, y.shape

AttributeError: module 'src.utils' has no attribute 'preprocessing_no_drop'

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
# create our classifier with scaling and desired hyperparameters
clf = SVM_model(kernel='rbf')
clf.fit(X_train, y_train)

score = clf.score(X_test, y_test)
print("Accuracy: ", score)


c:\Users\musta\anaconda3\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


Accuracy:  0.7222564102564103


In [5]:
# Grid search for SVM parameter C
from sklearn.model_selection import GridSearchCV

parameters = {'C':[0.1, 1, 10]}
clf = SVM_model(kernel='rbf')
grid_search = GridSearchCV(clf, parameters, verbose=10, n_jobs = -1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_, grid_search.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
{'C': 10} 0.7174727233274483


c:\Users\musta\anaconda3\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [6]:
# create our classifier with scaling and desired hyperparameters
clf = SVM_model(kernel='rbf', C=10)
clf.fit(X_train, y_train)

score = clf.score(X_test, y_test)
print(score)

c:\Users\musta\anaconda3\lib\site-packages\sklearn\svm\_base.py:299: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


0.7249230769230769


In [ ]:
clf.compare(model, X_test, y_test)